In [ ]:
import csv
import json
import argparse


# コマンドライン引数の受け取り-------------------------------------------------------------------------------
# parser = argparse.ArgumentParser(description="stable diffsuionでデータセットを生成するためのスクリプト")
# parser.add_argument("--CSVPATH",type=str, help="str: save dir name")
# args = parser.parse_args()
# CSVPATH = f"/root/dataset/SD_generated_dataset_checked/{args.CSVPATH}/annotations/annotations.csv"
CSVPATH = f"/root/dataset/original_dataset/annotations/annotations.csv"

# CSVファイルの読み込み
annotations = []
images = []
keypoints = []
image_id = 1

with open(CSVPATH, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        image_name = row[0]
        keypoints_converted = row[1:]

        keypoints = []
        for num,val in enumerate(keypoints_converted):
            keypoints.append(float(val))
            if num % 2 == 1:
                keypoints.append(2)

        # COCO keypoints形式に変換
        annotation = {
            "id": image_id,
            "image_id": image_id,
            "category_id": 1,
            "segmentation": [],
            "area": 0,  # この値は適切に計算する必要があります
            "bbox": [0,0,254,254],  # この値はkeypointsから計算する必要があります
            "iscrowd": 0,
            "attributes": {
                "occluded": False
            },
            "keypoints": keypoints,
            "num_keypoints": len(keypoints) // 3  # x, y, visibilityの3要素が1つのkeypoint
        }
        annotations.append(annotation)

        image = {
            "id": image_id,
            "width": 254,  # この値は実際の画像のサイズに合わせて変更する必要があります
            "height": 254,  # この値は実際の画像のサイズに合わせて変更する必要があります
            "file_name": image_name,
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        }
        images.append(image)

        image_id += 1

# COCO形式のJSONデータを作成
coco_data = {
    "licenses": [
        {
            "name": "",
            "id": 0,
            "url": ""
        }
    ],
    "info": {
        "contributor": "",
        "date_created": "",
        "description": "",
        "url": "",
        "version": "",
        "year": ""
    },
    "categories": [
        {
            "id": 1,
            "name": "keypoints",
            "supercategory": "",
            "keypoints": [str(i) for i in range(1, 73)],
            "skeleton": []
        }
    ],
    "images": images,
    "annotations": annotations
}

# JSONファイルに書き込み
output_file_dir = CSVPATH.replace("annotations.csv","")
output_file = 'output_coco_format.json'
with open(output_file_dir + output_file, 'w') as f:
    json.dump(coco_data, f, indent=4)


print(f"COCO Keypoints形式のデータを{output_file}に保存しました。")